In [1]:
# Установка необходимых пакетов для оценки 
# rouge: Для оценки с помощью метрики ROUGE
# bert_score: Для оценки с помощью BERTScore
!pip install rouge --quiet
!pip install bert_score --quiet

In [2]:
import os
import re
import pandas as pd

# Реализация метрики ROUGE на Python
from rouge import Rouge

# BERTScore использует предварительно обученные контекстные эмбеддинги BERT.
from bert_score import BERTScorer

# Задача резюмирования (суммаризации)

Будем использовать пример резюмирования ниже. Обратите внимание, что мы предоставляем два сгенерированных резюме для сравнения и референтное резюме, написанное человеком, которое требуется для таких метрик оценки, как ROUGE и BERTScore.

```
Миссия OpenAI заключается в том, чтобы искусственный интеллект общего назначения (AGI) приносил пользу всему человечеству. OpenAI будет создавать безопасный и полезный AGI напрямую, но также будет считать свою миссию выполненной, если его работа поможет другим достичь этого результата. Для достижения этой цели OpenAI следует нескольким ключевым принципам. Во-первых, широкое распределение благ - любое влияние на развертывание AGI будет использоваться на благо всех, чтобы избежать вредного использования или чрезмерной концентрации власти. Во-вторых, долгосрочная безопасность - OpenAI обязуется проводить исследования, чтобы сделать AGI безопасным, и способствовать внедрению этих исследований в сообществе ИИ. В-третьих, техническое лидерство - OpenAI стремится быть на переднем крае развития возможностей ИИ. В-четвертых, ориентация на сотрудничество - OpenAI активно сотрудничает с другими исследовательскими и политическими институтами и стремится к созданию глобального сообщества, работающего вместе над решением глобальных проблем AGI.
```

**Референсное саммари / ref_summary (создана человеком)**

```
OpenAI стремится к тому, чтобы искусственный интеллект общего назначения (ИИОН) использовался для всеобщего блага, не допуская вредного применения или чрезмерной концентрации власти. Она занимается исследованиями в области безопасности ИИ и продвигает эти исследования среди сообщества ИИ. OpenAI стремится к лидерству в развитии возможностей ИИ и сотрудничает с глобальными исследовательскими и политическими институтами для решения проблем AGI.
```

**Eval Summary 1 / eval_summary_1 (сгенерировано ML)**
```
OpenAI стремится к тому, чтобы AGI приносили пользу всему человечеству, избегая вредного использования и концентрации власти. OpenAI является первопроходцем в исследованиях безопасного и полезного AGI и способствует его внедрению во всем мире. OpenAI сохраняет техническое лидерство в области ИИ, сотрудничая с глобальными институтами для решения проблем AGI. Она стремится возглавить совместные мировые усилия по разработке AGI для коллективного блага.
```

**Eval Summary 2 / eval_summary_2 (сгенерировано ML)**
```
OpenAI стремится к тому, чтобы AGI использовался всеми, полностью исключая вредные вещи или концентрацию большой власти. OpenAI стремится исследовать безопасные стороны AGI и продвигать эти исследования среди пользователей ИИ. OpenAI хочет быть лучшим в области ИИ и работает с мировыми исследовательскими и политическими группами, чтобы понять, что такое AGI.
```

In [3]:
excerpt = "Миссия OpenAI заключается в том, чтобы искусственный интеллект общего назначения (AGI) приносил пользу всему человечеству. OpenAI будет создавать безопасный и полезный AGI напрямую, но также будет считать свою миссию выполненной, если его работа поможет другим достичь этого результата. Для достижения этой цели OpenAI следует нескольким ключевым принципам. Во-первых, широкое распределение благ - любое влияние на развертывание AGI будет использоваться на благо всех, чтобы избежать вредного использования или чрезмерной концентрации власти. Во-вторых, долгосрочная безопасность - OpenAI обязуется проводить исследования, чтобы сделать AGI безопасным, и способствовать внедрению этих исследований в сообществе ИИ. В-третьих, техническое лидерство - OpenAI стремится быть на переднем крае развития возможностей ИИ. В-четвертых, ориентация на сотрудничество - OpenAI активно сотрудничает с другими исследовательскими и политическими институтами и стремится к созданию глобального сообщества, работающего вместе над решением глобальных проблем AGI."
ref_summary = "OpenAI стремится к тому, чтобы искусственный интеллект общего назначения (ИИОН) использовался для всеобщего блага, не допуская вредного применения или чрезмерной концентрации власти. Она занимается исследованиями в области безопасности ИИ и продвигает эти исследования среди сообщества ИИ. OpenAI стремится к лидерству в развитии возможностей ИИ и сотрудничает с глобальными исследовательскими и политическими институтами для решения проблем AGI."
eval_summary_1 = "OpenAI стремится к тому, чтобы AGI приносили пользу всему человечеству, избегая вредного использования и концентрации власти. OpenAI является первопроходцем в исследованиях безопасного и полезного AGI и способствует его внедрению во всем мире. OpenAI сохраняет техническое лидерство в области ИИ, сотрудничая с глобальными институтами для решения проблем AGI. Она стремится возглавить совместные мировые усилия по разработке AGI для коллективного блага."
eval_summary_2 = "OpenAI стремится к тому, чтобы AGI использовался всеми, полностью исключая вредные вещи или концентрацию большой власти. OpenAI стремится исследовать безопасные стороны AGI и продвигать эти исследования среди пользователей ИИ. OpenAI хочет быть лучшим в области ИИ и работает с мировыми исследовательскими и политическими группами, чтобы понять, что такое AGI."

## Оценка с использованием ROUGE

ROUGE, что расшифровывается как Recall-Oriented Understudy for Gisting Evaluation, в первую очередь оценивает совпадение слов между сгенерированным результатом и эталонным текстом. Это распространенная метрика для оценки задач автоматического резюмирования. Среди ее разновидностей есть ROUGE-L, которая позволяет определить, насколько хорошо система сохраняет суть оригинального резюме, - это самое длинное непрерывное совпадение между сгенерированным системой и эталонным резюме.

In [4]:
# вычисление ROUGE_score
def get_rouge_scores(text1, text2):
    rouge = Rouge()
    return rouge.get_scores(text1, text2)


rouge_scores_out = []

# рассчет ROUGE для обоих резюме, используя референсное
eval_1_rouge = get_rouge_scores(eval_summary_1, ref_summary)
eval_2_rouge = get_rouge_scores(eval_summary_2, ref_summary)

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = eval_1_rouge[0][metric][label[0].lower()]
        eval_2_score = eval_2_rouge[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score,
            "Summary 2": eval_2_score,
        }
        rouge_scores_out.append(row)


def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]


rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
    .style.apply(highlight_max, axis=1)
)

rouge_scores_out

,Summary 1,Summary 2
Metric,,
rouge-1 (F-Score),0.395833,0.436782
rouge-2 (F-Score),0.200000,0.222222
rouge-l (F-Score),0.395833,0.436782


В таблице показаны оценки ROUGE для оценки двух различных резюме по сравнению с эталонным текстом. В случае с rouge-1 резюме 2 превосходит резюме 1, что свидетельствует о лучшем совпадении отдельных слов, и в случае с rouge-l резюме 2 имеет более высокий балл, что означает более тесное совпадение по самым длинным общим подпоследовательностям и, таким образом, потенциально лучшее общее обобщение, передающее основное содержание и порядок оригинального текста. Поскольку в резюме 2 много слов и коротких фраз, взятых непосредственно из отрывка, его совпадение с эталонным резюме, скорее всего, будет выше, что приведет к более высоким баллам ROUGE.

Хотя ROUGE и аналогичные метрики, такие как BLEU и METEOR, предлагают количественные показатели, они часто не отражают истинную суть хорошо составленного резюме. Кроме того, они хуже коррелируют с человеческими оценками. Учитывая развитие LLM, которые умеют создавать беглые и связные резюме, традиционные метрики, такие как ROUGE, могут невольно занижать баллы для этих моделей. Это особенно верно, если резюме сформулированы по-разному, но при этом точно отражают основную информацию.

## Оценка с использованием BERTScore

ROUGE опирается на точное присутствие слов в предсказанных и эталонных текстах, не умея интерпретировать лежащую в их основе семантику. Именно здесь на помощь приходит BERTScore, который использует контекстуальные эмбеддинги модели BERT, стремясь оценить сходство между предсказанным и эталонным предложением в контексте текста, созданного машиной. Сравнивая эмбеддинги из обоих предложений, BERTScore улавливает семантическое сходство, которое может быть упущено традиционными метриками, основанными на n-граммах.

In [5]:
# Создать экземпляр объекта BERTScorer для английского языка
scorer = BERTScorer(lang="ru")

# Рассчет BERTScore для резюме 1
# P1, R1, F1_1 представляют Precision, Recall, и F1 меру.
P1, R1, F1_1 = scorer.score([eval_summary_1], [ref_summary])

# Рассчет BERTScore для резюме 2
# P1, R1, F1_1 представляют Precision, Recall, и F1 меру.
P2, R2, F2_2 = scorer.score([eval_summary_2], [ref_summary])

print("Summary 1 F1 Score:", F1_1.tolist()[0])
print("Summary 2 F1 Score:", F2_2.tolist()[0])

Summary 1 F1 Score: 0.779007077217102
Summary 2 F1 Score: 0.8417514562606812


Близкие показатели F1 между резюме указывают на то, что они могут одинаково эффективно отражать ключевую информацию. Однако эту небольшую разницу следует интерпретировать с осторожностью. Поскольку BERTScore может не полностью улавливать тонкости и высокоуровневые понятия, которые может понять человек, опирающийся на эту метрику, может привести к неправильной интерпретации фактического качества и нюансов резюме. Интегрированный подход, сочетающий BERTScore с человеческим суждением и другими метриками, может предложить более надежную оценку.